## **Book Recommendation with a Target**

In this example, instead of using movie data, we use book ratings. This is essentially the same problem but the reason we use this example is because we will add a target to the recommender engine. This increases run-time as it creates a more complex model. 

In [1]:
import pandas as pd
import turicreate as tc
import requests
import io

In [2]:
from s3fs.core import S3FileSystem

## **Get the Data** 
We have uploaded the data to a public S3 pucket but the original data can be found [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

In [3]:
s3 = S3FileSystem(anon=True)
file = s3.open("s3://skafos.example.data/BX-Book-Ratings.csv", "r", encoding = 'latin1', errors='replace')

In [4]:
data = pd.read_csv(file, sep =";", error_bad_lines=False)

In [ ]:
data.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


## **Prep the data**
We convert the book-rating column to an integer value, convert the data to an SFrame (Turi Create's dataframe data structure) and split the training and validation data.

In [ ]:
# convert the book rating column to an integer
data['Book-Rating'] = data['Book-Rating'].astype(int)

In [ ]:
tc.config.set_num_gpus(0)

In [ ]:
# convert to an SFrame
data = tc.SFrame(data)

In [ ]:
# split the training and validation sets up
training_data, validation_data = tc.recommender.util.random_split_by_user(data, 'User-ID', 'ISBN')

## **Build the model**
Here we build the model. Note how this example differs from the pre-baked Turi Create example in that we specify a target. For more information about this, checkout the [Turi Create documentation](https://turi.com/learn/userguide/recommender/choosing-a-model.html)

In [ ]:

# build the recommender
#model = tc.recommender.create(training_data, 'User-ID', 'ISBN', target = 'Book-Rating')
model = tc.recommender.create(observation_data=training_data, user_id = 'User-ID', item_id='ISBN', target = 'Book-Rating' )

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 1147266 observations with 105171 users and 340163 items.

Data prepared in: 0.463705s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 7.28687

Final training RMSE: 1.40293

In [ ]:
# Check Results

In [ ]:

# grab the results of the model
results = model.recommend();

recommendations finished on 1000/105171 queries. users per second: 132.324

In [ ]:
results

In [ ]:
model.evaluate(validation_data)